In [49]:
%cd ..

/Users/treycole/Codes/WanPy


In [53]:
from WanPy.WanPy import *
import WanPy.models as models
import WanPy.plotting as plot

from pythtb import *
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sympy as sp 
import scipy

In [68]:
delta = 1
t0 = 0.27
tprime = 0.5

model = models.chessboard(t0, tprime, delta).make_supercell([[2,0], [0,2]])

orbs = model.get_orb()
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)
lat_vecs = model.get_lat() # lattice vectors

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)

u_wfs_full = wf_array(model, [20, 20])
u_wfs_full.solve_on_grid([0, 0])
chern = u_wfs_full.berry_flux([i for i in range(n_occ)])/(2*np.pi)

print(f"Chern number: {chern: .2f}")

Chern number: -1.00


In [76]:
nkx = 10
nky = 10
Nk = nkx*nky

k_mesh = gen_k_mesh(nkx, nky, flat=False, endpoint=False)
k_mesh_flat = gen_k_mesh(nkx, nky, flat=True, endpoint=False)

u_wfs_Wan = wf_array(model, [nkx, nky])
for i in range(k_mesh.shape[0]):
    for j in range(k_mesh.shape[1]):
        u_wfs_Wan.solve_on_one_point(k_mesh[i,j], [i,j])
psi_wfs_Wan = get_bloch_wfs(orbs, u_wfs_Wan, k_mesh)

In [82]:
omit_sites = None
tf_list = list(np.setdiff1d(low_E_sites, [omit_sites])) # delta on lower energy sites omitting the last site

W0, psi_til_wan = Wannierize(orbs, u_wfs_Wan, tf_list, ret_psi_til=True)
u_tilde_wan = get_bloch_wfs(orbs, psi_til_wan, k_mesh, inverse=True)

In [83]:
state_idx = list(range(0, n_occ))
tfs = set_trial_function(tf_list, n_orb)
A = tf_overlap_mat(psi_wfs_Wan, tfs, state_idx)
V, S, Wh = SVD(A, full_matrices=False)

In [84]:
sp.Matrix(A[0,0].round(10))

Matrix([
[         -0.5,                           -0.5,                          -0.5,          -0.5],
[ 0.1849870904, -0.5835113296 + 0.1401137821*I, 0.5835113296 - 0.1401137821*I, -0.1849870904],
[-0.6000977784, -0.1798741254 + 0.0431916961*I, 0.1798741254 - 0.0431916961*I,  0.6000977784],
[            0,                              0,                             0,             0]])

In [86]:
sp.Matrix(np.diag(S[0,0]).round(10))

Matrix([
[1.0,           0,           0, 0],
[  0, 0.888073834,           0, 0],
[  0,           0, 0.888073834, 0],
[  0,           0,           0, 0]])

In [71]:
A = np.random.rand(5000, 5000)
# sp.Matrix(A)

In [45]:
V, S, Wh = np.linalg.svd(A)
V, S, Wh = V[:, :-1], S[:-1], Wh[:-1, :]

In [46]:
A_prod = V@np.diag(S)@Wh
diff = A-A_prod
# sp.Matrix((diff).round(10))

In [47]:
np.linalg.norm(diff)

0.000536516934817116